In [92]:
import torch
import torchviz
import os
import torch.nn as nn
import pandas as pd
import numpy as np      
from sklearn.model_selection import train_test_split
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.utils.data import WeightedRandomSampler, RandomSampler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, accuracy_score, precision_recall_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import auc
# import libraries ต่าง ๆ ที่จะใช้
import torchviz
import torch
import os
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # ใช้ในการแบ่งข้อมูล
import torch.nn as nn
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot
from torchmetrics.classification import MulticlassPrecisionRecallCurve
from torch.utils.data import WeightedRandomSampler
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from optuna.integration.tensorboard import TensorBoardCallback
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import KFold

In [93]:
# import dataset
df = pd.read_csv("../3 - Cleaning & Transforming Data/2 cleaned_dataset_MERGED_DRUG(Mogran)_ALLELESEQ.csv", index_col = None)

if(df.isna().any().sum()) != 0:
    raise Exception('There are still missing values in the dataset!')

In [94]:
# check ของแปลกที่หลุดเข้ามา
for x in df.columns:
    print(x)

Drug
Allele
ADR_MPexanthema
ADR_SJS
ADR_TEN
ADR
Cohort ethnicity_Black
Cohort ethnicity_Caucasian
Cohort ethnicity_Diverse
Cohort ethnicity_Mongol
Cohort ethnicity
No_ADR
Yes_ADR
0
1
2
3
4
11
20
23
30
74
80
86
122
123
130
140
151
162
175
191
204
207
245
248
264
294
297
300
314
315
319
323
333
336
338
350
352
366
371
378
381
389
405
409
426
428
435
448
456
461
469
470
485
486
501
502
511
527
530
535
547
554
556
561
588
600
609
623
624
631
637
650
656
675
679
694
713
715
725
739
745
750
751
764
770
772
780
787
794
798
806
807
833
834
843
849
872
875
882
883
884
888
926
935
952
974
984
987
990
993
1011
1017
1019
1028
1039
1043
1044
1055
1057
1060
1066
1070
1077
1088
1097
1098
1099
1102
1105
1114
1120
1130
1137
1138
1142
1145
1148
1152
1153
1155
1164
1168
1171
1182
1184
1199
1212
1236
1242
1267
1269
1270
1292
1295
1312
1313
1318
1325
1340
1347
1351
1357
1380
1389
1391
1398
1426
1428
1434
1446
1448
1449
1452
1457
1459
1464
1476
1480
1498
1528
1535
1542
1544
1564
1578
1585
1589
1602
1607
164

In [95]:
df['ADR'].value_counts()

SJS            5805
TEN            3738
MPexanthema    3530
Name: ADR, dtype: int64

In [96]:
# Drop unnecessary columns
df_stratify = pd.DataFrame()
df['ADR'] = df['ADR'].replace({'SJS': 0, 'TEN': 1, 'MPexanthema':2})#,'MPeruption':3})
#df = df[df['ADR'].isin([0,1])] #ลบ MP ออก
#df_stratify['Yes_ADR'] = df['ADR'].copy()
#df_stratify = df_stratify.rename(columns={'Yes_ADR':'ADR'})
df.drop(columns=['No_ADR','ADR','ADR_MPexanthema','ADR_SJS','ADR_TEN','Cohort ethnicity','Allele','Drug'], inplace=True) #'MPeruption' removed

In [97]:
# Select output columns and drop unnecessary columns
df_output = df['Yes_ADR'].copy()
df.drop(columns = ['Yes_ADR'], inplace = True)
df_input = df.copy()
# Check input dtypes
df_input.dtypes.value_counts()

int64    350
dtype: int64

In [98]:
# check ของแปลกที่หลุดเข้ามา
df_output

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
13068    0.0
13069    0.0
13070    0.0
13071    0.0
13072    0.0
Name: Yes_ADR, Length: 13073, dtype: float64

In [99]:
# check ของแปลกที่หลุดเข้ามา
for x in df_input.columns:
    print(x)

Cohort ethnicity_Black
Cohort ethnicity_Caucasian
Cohort ethnicity_Diverse
Cohort ethnicity_Mongol
0
1
2
3
4
11
20
23
30
74
80
86
122
123
130
140
151
162
175
191
204
207
245
248
264
294
297
300
314
315
319
323
333
336
338
350
352
366
371
378
381
389
405
409
426
428
435
448
456
461
469
470
485
486
501
502
511
527
530
535
547
554
556
561
588
600
609
623
624
631
637
650
656
675
679
694
713
715
725
739
745
750
751
764
770
772
780
787
794
798
806
807
833
834
843
849
872
875
882
883
884
888
926
935
952
974
984
987
990
993
1011
1017
1019
1028
1039
1043
1044
1055
1057
1060
1066
1070
1077
1088
1097
1098
1099
1102
1105
1114
1120
1130
1137
1138
1142
1145
1148
1152
1153
1155
1164
1168
1171
1182
1184
1199
1212
1236
1242
1267
1269
1270
1292
1295
1312
1313
1318
1325
1340
1347
1351
1357
1380
1389
1391
1398
1426
1428
1434
1446
1448
1449
1452
1457
1459
1464
1476
1480
1498
1528
1535
1542
1544
1564
1578
1585
1589
1602
1607
1646
1647
1657
1675
1683
1708
1722
1738
1739
1742
1750
1778
1786
1803
1804
1816
182

In [100]:
# # Change all input dtypes to float64
# df_input = df_input.astype('float64')
# df_input.dtypes.value_counts()

In [101]:
# Convert to numpy array
np_input = df_input.to_numpy()
np_output = df_output.to_numpy()
np_stratify = df_stratify.to_numpy()

In [102]:
pd.DataFrame(np_stratify).value_counts()

Series([], dtype: int64)

In [103]:
np_input    

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [104]:
# Function to check dataloader classes distribution
def get_dataloader_distribution(*args, label = None):
    graph_no = len(args)
    fig, ax = plt.subplots(graph_no,1)
    
    for index, dataloader in enumerate(args):
        batch_no = [x+1 for x in range(len(dataloader))]
        class_0_count = []
        class_1_count = []
        batch_count = []

        for i, (data,target) in enumerate(dataloader):
            tocount = pd.DataFrame(target.numpy()).value_counts()
            batch_count.append(i+1)
            try:
                class_0_count.append(tocount[0])
            except:
                class_0_count.append(0)
            try:
                class_1_count.append(tocount[1])
            except:
                class_1_count.append(0)
        X_axis = np.arange(len(batch_count))

        
        fig.set_figwidth(15)
        fig.set_figheight(10)
        ax[index].set_xlabel="Batch Number"
        ax[index].set_ylabel="No. Datapoints"
        ax[index].bar(X_axis+(1/3),class_0_count,width=(1/3) ,label = "No ADR",color='red',alpha=0.5)
        ax[index].bar(X_axis,class_1_count,width=(1/3), label = 'Yes ADR',color='green',alpha=0.5)
        #ax.set_xticks(X_axis+width)
        ax[index].legend()
    
    ax[0].set_title(f"{label} Train_Loader Label Distribution by Yes/No ADR (per batch)")
    ax[1].set_title(f"{label} Test_loader Label Distribution by Yes/No ADR")
    ax[1].set_xticks([])
    plt.show()

In [105]:
print(np_input.shape)
print(np_output.shape)

(13073, 350)
(13073,)


In [106]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    #print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

## Define Model

In [107]:
# Define Network Classes
class Network(nn.Module):
    def __init__(self, n_layers, n_first_units, dropout):
        super().__init__()

    # Define Layers    
        layers = []
        in_features = np_input.shape[1]
        out_features = n_first_units 
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        in_features = out_features
        for i in range(n_layers): #i หมายถึงเลขชั้น hidden+1

            layers.append(nn.BatchNorm1d(in_features))
            out_features = int(in_features/2)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_features = out_features
        layers.append(nn.BatchNorm1d(out_features))
        layers.append(nn.Linear(out_features, 1)) 
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)

    # Define the forward pass
    def forward(self,input):
        output = self.layers(input)
        return output


In [108]:
# Empty CUDA cache
torch.cuda.empty_cache()

# Assign device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [109]:
# Define train function

def train(dataloader, model, loss_fn, optimizer):

    
    # Running Loss and Train Accuracy 
    running_loss = []
    running_train_acc = []

    # Set model to train mode
    model.train()

    # Loop through dataloader
    for batch_idx, (data, target) in enumerate(dataloader):

        # Move data to device
        data = data.float().to(device)
        target = target.float().to(device)
        
        # Save Model Graph
        # if batch_idx == 0:
        #     dot = torchviz.make_dot(model(data), show_attrs=True, show_saved = True,params=dict(model.named_parameters()))
        #     dot.render(f'{dir}/{name}')

        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        
        # Calculate loss
        output = output.flatten()         
        loss = loss_fn(output,target)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Save loss for each batch
        running_loss.append(loss.item()) 

        # Calculate accuracy
        with torch.no_grad():
            target = target.flatten()
            output = torch.round(output)
            correct = (output == target).sum().item()
            train_acc = correct/len(target)
            train_acc = round(train_acc, 8)
            running_train_acc.append(train_acc)

    train_acc = np.mean(running_train_acc)
    train_loss = np.mean(running_loss)
    #print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    return float(train_loss), float(train_acc)

In [110]:
# Define test function

def test(dataloader, model, loss_fn):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):

            # Move data to device
            data = data.float().to(device)
            target = target.float().to(device)    
            
            # Forward pass           
            output = model(data)

            # Calculate loss
            output = output.flatten()
            data = data.flatten()
            
            test_loss = loss_fn(output, target)

            # Calculate accuracy
            output_notrounded = output
            output = torch.round(output)
            correct = (output == target).sum().item()
        output_notrounded = output_notrounded.cpu().detach().numpy()
        output = output.cpu().detach().numpy()
        target = target.cpu().detach().numpy()

        # Calculate metrics
        test_acc = correct / len(dataloader.dataset)
        test_loss = test_loss.cpu().detach().numpy()
        test_precision = precision_score(output, target, zero_division=0)
        test_f1_score = f1_score(output, target, zero_division=0)
        test_recall_score = recall_score(output, target, zero_division=0)

        # precision, recall, thresholds = precision_recall_curve(target, output_notrounded)
        # F1Score =  (2*precision*recall) / (precision + recall)
        # max_F1Score = np.max(F1Score)
        # max_index_F1Score = np.argmax(F1Score)
        # test_auc_score = auc(recall, precision)
    #print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall_score:.4f}, Test F1 Score: {test_f1_score:.4f}")

    return float(test_loss), float(test_acc), float(test_precision), float(test_recall_score), float(test_f1_score)#, float(test_auc_score), float(max_F1Score), float(thresholds[max_index_F1Score])

# Adding Optuna

In [111]:
# Define NOT Optimized Hyperparameters

#epochs = 100
# L2lambda = 0.01
k_folds = 5
DIR = os.getcwd()

loss_fn = nn.BCELoss() # Binary Cross Entropy Loss

epochs = 800
dirname = "optuna_run2" 
batch_size = 128
learning_rate = 0.01
n_layers = 2
n_first_units = 512

In [ ]:
# Default Values
L2lambda = 0.01
dropout = 0.2

In [113]:
def define_model(trial):
    #global epochs
    #epochs = trial.suggest_int("epochs", 100, 1000, step = 100)
    #epochs=1
    # Optimize Hyperparameters
    YorN_batchnorm = trial.suggest_categorical("batch_norm",['yes','no'])
    global YorN_dropout
    global dropout
    YorN_dropout = trial.suggest_categorical("dropout",['yes','no'])
    #if YorN_dropout == 'yes':
    #    dropout = trial.suggest_float("dropout_l", 0.1, 0.4, step = 0.1) 
    #n_layers = trial.suggest_int("n_layers", 1,5)
    #n_first_units = trial.suggest_categorical("n_first_units", [128,256,512,1024])


    # Define Model
    layers = []

    # Input Layer
    in_features = np_input.shape[1]
    out_features = n_first_units
    layers.append(nn.Linear(in_features, out_features))
    layers.append(nn.ReLU())
    if YorN_dropout == 'yes':
        layers.append(nn.Dropout(dropout))
    in_features = out_features

    # Hidden Layers
    for i in range(n_layers): #i หมายถึงเลขชั้น hidden+1
        #out_features = trial.suggest_int("n_units_l{}".format(i), 5, 4096) 
        #out_features = trial.suggest_int("n_units_l{}".format(i), 256, 2048, 256) 
        if YorN_batchnorm == 'yes':
            layers.append(nn.BatchNorm1d(in_features))
        out_features = int(in_features/2)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        if YorN_dropout == 'yes':
            layers.append(nn.Dropout(dropout))
        in_features = out_features

    # Output Layer
    if YorN_batchnorm == 'yes':
        layers.append(nn.BatchNorm1d(in_features))                          
    layers.append(nn.Linear(out_features, 1)) 
    layers.append(nn.Sigmoid())

    # Return model
    return nn.Sequential(*layers)


def objective(trial):

    # Optimize Hyperparameters
    #batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512, 1024, 2048])
    YorN_weightdecay = trial.suggest_categorical("weightdecay",['yes','no'])
    
    # Create model
    model = define_model(trial).to(device)
    #print(model)

    # Assign Optimizer
    #learning_rate = trial.suggest_categorical("learning_rate", [1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
    #learning_rate = 1e-3
    optimizer_name = "Adam"
    if YorN_weightdecay == 'yes':
        L2lambda = trial.suggest_categorical("L2lambda", [1e-4, 1e-3, 1e-2, 1e-1, 1])
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate, weight_decay=L2lambda) #เพิ่ม weight decay เพื่อให้ model ไม่ overfit
    else:
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    # Assign Run Name
    name = f"trial-{trial.number}"
    dir = os.path.join(dirname, name)

    # Create K-Fold and lock Random Seed
    kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=123)
    torch.manual_seed(123)

    total_train_loss = []
    total_test_loss = []
    total_train_acc = []
    total_test_acc = []
    total_test_f1 = []
    total_test_precision = []
    total_test_recall = []
    total_test_auc = []

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(np_input,np_output)):
        
        # Reset model   
        model.apply(reset_weights)

        fold_train_loss = []
        fold_test_loss = []
        fold_train_acc = []
        fold_test_acc = []
        fold_test_f1 = []
        fold_test_precision = []
        fold_test_recall = []
        #fold_test_auc = []

        # Check disjoint for each fold จ้า
        for x in train_ids: 
            if x in test_ids:
                raise Exception("WARNING: Train and Test IDs are not disjoint!")
            
        print(f"Trial {trial.number} - Fold {fold} - Params: {trial.params}")
        if YorN_weightdecay == 'yes':
            print(f"Additional Hyperparameters -> Weight Decay: {L2lambda}")
        if YorN_dropout == 'yes':
            print(f"Additional Hyperparameters -> Dropout: {dropout}")
        # Print Fold
        print(f'FOLD {fold}')   
        print(f'--------')   

        # Create data for each fold with the index provided by kfold.split
        np_fold_train_input = np_input[train_ids]
        np_fold_train_output = np_output[train_ids]
        np_fold_test_input = np_input[test_ids]
        np_fold_test_output = np_output[test_ids]

        # Create Weight for Sampler
        weight = (1/pd.DataFrame(np_fold_train_output).value_counts()).tolist()
        sample_weights = np.array([weight[int(t)] for t in np_fold_train_output])

        # Create Sampler
        sampler = WeightedRandomSampler(weights=sample_weights,num_samples=len(sample_weights), replacement=True)
        #train_sampler = RandomSampler(np_fold_train_output) #RandomSampler จะทำให้เห็นว่า class มัน imbalance ยังไง

        # Create Dataset and Dataloader
        train_dataset = TensorDataset(torch.from_numpy(np_fold_train_input).float(), torch.from_numpy(np_fold_train_output).float())
        test_dataset = TensorDataset(torch.from_numpy(np_fold_test_input).float(), torch.from_numpy(np_fold_test_output).float())
        train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
        test_loader = DataLoader(test_dataset, batch_size=len(test_ids))
        #get_dataloader_distribution(train_loader,test_loader,label=f'Fold {fold}') #ใช้เช็ค Distribution ของแต่ละ Fold


        # Train / Test Loops
        for t in range(epochs):
            
            train_loss, train_acc = train(train_loader, model, loss_fn, optimizer)
            test_loss, test_acc, test_precision, test_recall, test_f1 = test(test_loader, model, loss_fn) # test_auc_score, test_maxF1, test_bestthershold 
            print(f"epoch: {t}, train loss: {train_loss:.4f}, test loss: {test_loss:.4f}, train acc: {train_acc:.4f}, test acc: {test_acc:.4f}, test precision: {test_precision:.4f}, test recall: {test_recall:.4f}, test f1: {test_f1:.4f}")# test auc: {test_auc_score:.4f}, test maxF1: {test_maxF1:.4f}, test best threshold: {test_bestthershold:.4f}")
            fold_train_loss.append(train_loss)
            fold_test_loss.append(test_loss)
            fold_train_acc.append(train_acc)
            fold_test_acc.append(test_acc)
            fold_test_f1.append(test_f1)
            fold_test_precision.append(test_precision)
            fold_test_recall.append(test_recall)
            # fold_test_auc.append(test_auc_score)
            with tf.summary.create_file_writer(f"{dir}_train_fold{fold}").as_default():
                tf.summary.scalar("loss", train_loss, step=t)
                tf.summary.scalar("acc", train_acc, step=t)
            with tf.summary.create_file_writer(f"{dir}_test_fold{fold}").as_default():    
                tf.summary.scalar("loss", test_loss, step=t) 
                tf.summary.scalar("acc", test_acc, step=t)
                tf.summary.scalar("precision", test_precision, step=t)
                tf.summary.scalar("f1_score", test_f1, step=t)
                tf.summary.scalar("recall_score", test_recall, step=t)
                # tf.summary.scalar("auc_score", test_auc_score, step=t)
                # tf.summary.scalar("auc_score", test_maxF1, step=t)

        # Added last value of each fold to total list
        total_train_loss.append(train_loss)
        total_test_loss.append(test_loss)
        total_train_acc.append(train_acc)
        total_test_acc.append(test_acc)
        total_test_f1.append(test_f1)
        total_test_precision.append(test_precision)
        total_test_recall.append(test_recall)
        # total_test_auc.append(test_auc_score)

        trial.set_user_attr(f"{fold} Train_loss: ", train_loss)
        trial.set_user_attr(f"{fold} Test_loss: ", test_loss)
        trial.set_user_attr(f"{fold} Train_acc: ", train_acc)
        trial.set_user_attr(f"{fold} Test_acc: ", test_acc)
        trial.set_user_attr(f"{fold} Test_precision: ", test_precision)
        trial.set_user_attr(f"{fold} Test_recall: ", test_recall)
        trial.set_user_attr(f"{fold} Test_f1: ", test_f1)
        # trial.set_user_attr(f"{fold} Test_auc: ", test_auc_score)
        # trial.set_user_attr(f"{fold} Test_maxF1: ", test_maxF1)
        # trial.set_user_attr(f"{fold} Test_bestthreshold: ", test_bestthershold)

        trial.report(test_f1, t)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    print("Training Done")
    print(f"K-FOLD CROSS VALIDATION FOR {k_folds} FOLDS, IN {epochs} EPOCHS")
    print(f"Total Train Loss: {np.mean(total_train_loss):.5f}")
    print(f"Total Test Loss: {np.mean(total_test_loss):.5f}")
    print(f"Total Train Acc: {np.mean(total_train_acc):.5f}")
    print(f"Total Test Acc: {np.mean(total_test_acc):.5f}")
    print(f"Total Test Precision: {np.mean(total_test_precision):.5f}")
    print(f"Total Test Recall: {np.mean(total_test_recall):.5f}")
    print(f"Total Test F1: {np.mean(total_test_f1):.5f}")
    print(f"Total Test AUC: {np.mean(total_test_auc):.5f}")

    trial.set_user_attr(f"Total Train_loss", np.mean(total_train_loss))
    trial.set_user_attr(f"Total Test_loss", np.mean(total_test_loss))
    trial.set_user_attr(f"Total Train_acc",np.mean(total_train_acc))
    trial.set_user_attr(f"Total Test_acc", np.mean(total_test_acc))
    trial.set_user_attr(f"Total Test_precision" , np.mean(total_test_precision))
    trial.set_user_attr(f"Total Test_recall" , np.mean(total_test_recall))
    trial.set_user_attr(f"Total Test_f1" , np.mean(total_test_f1))
    trial.set_user_attr(f"Total Test_auc" , np.mean(total_test_auc))

    try:
        fig,ax = plt.subplots()
        X_axis = np.arange(1,k_folds+1,1)
        ax.bar(X_axis,total_test_f1, color='green',label = "F1 Score",width=1/4)
        ax.bar(X_axis-(1/4),total_test_precision, color='blue',label = "Precision",width=1/4)
        ax.bar(X_axis-(2/4),total_test_recall, color='red',label = "Recall",width=1/4)
        ax.bar(X_axis-(3/4),total_test_acc, color='yellow',label = "Accuracy",width=1/4)
        ax.set_xlabel("Fold")
        ax.legend()
        ax.set_ylabel("F1 Score")
        plt.tight_layout()
        plt.savefig(f"../4 - Training & Testing/{dirname}/{name}_metrics.png")
    except:
        print("Cannot save metrics plot, passed to continue")
        pass

    return np.mean(total_test_f1)

if __name__ == "__main__":
    
    tensorboard_callback = TensorBoardCallback(dirname = dirname, metric_name="target")
    #pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3)
    pruner = optuna.pruners.HyperbandPruner(min_resource=1, max_resource=1000, reduction_factor=3)
    study = optuna.create_study(study_name = dirname, directions=["maximize"],sampler=TPESampler(),storage='sqlite:///optuna.sqlite3',pruner=pruner, load_if_exists=True)
    study.optimize(objective, n_trials=1, timeout=None)
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:231: ExperimentalWarning: TensorBoardCallback is experimental (supported from v2.0.0). The interface can change in the future.
[I 2023-04-20 11:53:01,122] Using an existing study with name 'optuna_run2' instead of creating a new one.


Trial 1 - Fold 0 - Params: {'weightdecay': 'yes', 'batch_norm': 'no', 'dropout': 'no', 'L2lambda': 0.1}
Additional Hyperparameters -> Weight Decay: 0.1
FOLD 0
--------
epoch: 0, train loss: 0.6932, test loss: 0.6899, train acc: 0.5027, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 1, train loss: 0.6933, test loss: 0.6923, train acc: 0.4988, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 2, train loss: 0.6932, test loss: 0.6897, train acc: 0.5035, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 3, train loss: 0.6933, test loss: 0.6895, train acc: 0.4927, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 4, train loss: 0.6932, test loss: 0.6935, train acc: 0.4955, test acc: 0.3545, test precision: 1.0000, test recall: 0.3545, test f1: 0.5234
epoch: 5, train loss: 0.6932, test loss: 0.6882, train acc: 0.5053, test acc: 0.6455, test precis

c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\optuna\trial\_trial.py:492: UserWarning: The reported value is ignored because this `step` 799 is already reported.
  step


epoch: 0, train loss: 0.6933, test loss: 0.6984, train acc: 0.5003, test acc: 0.3545, test precision: 1.0000, test recall: 0.3545, test f1: 0.5234
epoch: 1, train loss: 0.6933, test loss: 0.6933, train acc: 0.4976, test acc: 0.3545, test precision: 1.0000, test recall: 0.3545, test f1: 0.5234
epoch: 2, train loss: 0.6932, test loss: 0.6873, train acc: 0.5042, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 3, train loss: 0.6931, test loss: 0.6968, train acc: 0.5061, test acc: 0.3545, test precision: 1.0000, test recall: 0.3545, test f1: 0.5234
epoch: 4, train loss: 0.6932, test loss: 0.6913, train acc: 0.5065, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 5, train loss: 0.6932, test loss: 0.6918, train acc: 0.4950, test acc: 0.6455, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 6, train loss: 0.6931, test loss: 0.6928, train acc: 0.5073, test acc: 0.6455, test precision: 0.0000, test rec

c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\optuna\trial\_trial.py:492: UserWarning: The reported value is ignored because this `step` 799 is already reported.
  step


epoch: 0, train loss: 0.6933, test loss: 0.6954, train acc: 0.5018, test acc: 0.3546, test precision: 1.0000, test recall: 0.3546, test f1: 0.5236
epoch: 1, train loss: 0.6932, test loss: 0.6926, train acc: 0.5003, test acc: 0.6454, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 2, train loss: 0.6932, test loss: 0.6937, train acc: 0.4966, test acc: 0.3546, test precision: 1.0000, test recall: 0.3546, test f1: 0.5236
epoch: 3, train loss: 0.6933, test loss: 0.6941, train acc: 0.4972, test acc: 0.3546, test precision: 1.0000, test recall: 0.3546, test f1: 0.5236
epoch: 4, train loss: 0.6931, test loss: 0.6978, train acc: 0.5087, test acc: 0.3546, test precision: 1.0000, test recall: 0.3546, test f1: 0.5236
epoch: 5, train loss: 0.6933, test loss: 0.6901, train acc: 0.5011, test acc: 0.6454, test precision: 0.0000, test recall: 0.0000, test f1: 0.0000
epoch: 6, train loss: 0.6932, test loss: 0.6977, train acc: 0.5039, test acc: 0.3546, test precision: 1.0000, test rec

[W 2023-04-20 12:25:06,523] Trial 1 failed with parameters: {'weightdecay': 'yes', 'batch_norm': 'no', 'dropout': 'no', 'L2lambda': 0.1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\harit\AppData\Local\Temp\ipykernel_26196\134380466.py", line 142, in objective
    train_loss, train_acc = train(train_loader, model, loss_fn, optimizer)
  File "C:\Users\harit\AppData\Local\Temp\ipykernel_26196\1077543546.py", line 14, in train
    for batch_idx, (data, target) in enumerate(dataloader):
  File "c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\utils\data\dataloader.py", line 628, in __next__
    data = self._next_data()
  File "c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\utils\data\dataloader.py", line 671,

epoch: 402, train loss: 0.6932, test loss: 0.6949, train acc: 0.4989, test acc: 0.3546, test precision: 1.0000, test recall: 0.3546, test f1: 0.5236


KeyboardInterrupt: 